## Лабораторная работа № 3. DBSCAN

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import Literal
from warnings import filterwarnings

filterwarnings('ignore')

Необходимые функции и классы:

In [2]:
def hist(df: pd.DataFrame, title_text: str): 
    BAR_COLORS = ['#D9A23E', '#C47451', '#D9A23E', '#6B8E7A', '#C47451', '#99A3B8'] * 2 
    BAR_COLORS = BAR_COLORS[:len(df.columns)] 

    PLOT_BG_COLOR = '#2D2A26'    
    AXIS_COLOR = '#D9CAB3'       
    GRID_COLOR = '#736E66'   
    TITLE_FONT_COLOR = 'white'

    AXIS_LABELS = {'yaxis_title': 'Частота'}

    ROWS = 4
    COLS = 3
    N_PLOTS = len(df.columns) 

    fig = make_subplots(
        rows=ROWS, 
        cols=COLS, 
        horizontal_spacing=0.05, 
        vertical_spacing=0.08,
        subplot_titles=[f'Распределение {col}' for col in df.columns]
    )

    for i, col in enumerate(df.columns):

        row = (i // COLS) + 1
        col_idx = (i % COLS) + 1
        
        current_color = BAR_COLORS[i]

        hist_trace = go.Histogram(
            x=df[col],
            name=col,
            marker=dict(
                color=current_color,
                line=dict(width=0.5, color=AXIS_COLOR) 
            ),
            xbins=dict(size=1.0) 
        )

        fig.add_trace(hist_trace, row=row, col=col_idx)

    fig.update_layout(
        title={
            'text': title_text,
            'y': 0.98,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 18, 'color': TITLE_FONT_COLOR} 
        },
        showlegend=False,
        autosize=True,
        height=800,
        plot_bgcolor=PLOT_BG_COLOR, 
        paper_bgcolor=PLOT_BG_COLOR, 
        font_color=AXIS_COLOR, 
        font=dict(color=AXIS_COLOR) 
    )

    for i in range(1, N_PLOTS + 1):
        r = (i - 1) // COLS + 1
        c = (i - 1) % COLS + 1
        
        fig.update_xaxes(
            color=AXIS_COLOR,
            gridcolor=GRID_COLOR,
            row=r, col=c
        )
        
        fig.update_yaxes(
            title_text=AXIS_LABELS['yaxis_title'] if c == 1 else None,
            color=AXIS_COLOR,
            gridcolor=GRID_COLOR,
            row=r, col=c
        )

    fig.show()



def boxplots(df):

    PLOT_BG_COLOR = '#2D2A26'
    AXIS_COLOR = '#D9CAB3'
    TITLE_COLOR = 'white'

    df_melted = df.melt(var_name='Переменная', value_name='Значение')

    fig = px.box(
        df_melted,
        x='Переменная',
        y='Значение',
        title='Ящик с усами',
        color_discrete_sequence=['#D9A23E'] 
    )

    fig.update_layout(
        title={
            'text': 'Ящик с усами',
            'x': 0.5,
            'font': {'size': 18, 'color': TITLE_COLOR}
        },
        height=500,
        width=600,
        plot_bgcolor=PLOT_BG_COLOR,
        paper_bgcolor=PLOT_BG_COLOR,
        font=dict(color=AXIS_COLOR),
        xaxis=dict(
            tickfont=dict(color=AXIS_COLOR),
            linecolor=AXIS_COLOR,
            gridcolor='#736E66'
        ),
        yaxis=dict(
            tickfont=dict(color=AXIS_COLOR),
            linecolor=AXIS_COLOR,
            gridcolor='#736E66'
        )
    )

    fig.show()



def scale(df, scaler: str = 'Standard'):
    if scaler == 'Standard':
        model = StandardScaler().fit(df)
    else:
        model = MinMaxScaler().fit(df)
        
    scaled_df = model.transform(df)

    return pd.DataFrame(scaled_df, columns=df.columns, index=df.index)



class dbscan_clustering:

    def __init__(self, df: pd.DataFrame, eps: float = 0.5, min_samples: int = 5, metric: str = 'euclidean', 
                 algorithm: str = 'auto', p=None, origin_df: pd.DataFrame = None):
        
        self.df = df
        self._df_result = None
        self.eps = eps
        self.min_samples = min_samples
        self.metric = metric
        self.algorithm = algorithm
        self.p = p
        self.origin_df = origin_df

        self.clustering()


    def clustering(self):
        self.dbscan = DBSCAN(eps=self.eps, min_samples=self.min_samples, metric=self.metric, 
                             algorithm=self.algorithm, p=self.p).fit(self.df)
        
        if self.origin_df is None:
            self._df_result = self.df.copy()
        
        else:
            self._df_result = self.origin_df.copy()

        self._df_result['cluster'] = self.dbscan.labels_
        
        return self._df_result


    def scatter_plot(self, color='earth'):
        if self._df_result is None:
            self.clustering() 

        PLOT_BG_COLOR = '#2D2A26'
        AXIS_COLOR = '#D9CAB3'
        
        dimensions = self._df_result.columns[:-1]

        fig = px.scatter_matrix(
            self._df_result,
            dimensions=dimensions,
            color='cluster',
            title=f'Матрица рассеяния оценок DBSCAN (eps={self.eps}, min_samples={self.min_samples})',
            height=750, 
            width=850,
            color_continuous_scale=color
        )

        fig.update_traces(
            diagonal_visible=False,
            marker=dict(size=5, opacity=0.8),
            showupperhalf=False
        )

        fig.update_layout(
            title_x=0.5,
            paper_bgcolor=PLOT_BG_COLOR,
            plot_bgcolor=PLOT_BG_COLOR,
            font=dict(color=AXIS_COLOR)
        )

        for annotation in fig.layout.annotations:
            annotation.font.color = AXIS_COLOR

        fig.show()


    def mean_by_cluster(self):
        df_result = self._df_result

        return df_result.groupby('cluster').mean()
    
    def cluster_size(self):
        df_result = self._df_result

        return df_result.groupby('cluster').size()


    def grid_search(self, eps_range: int|float = 7, eps_step: int|float = 0.5, min_samples_range: int = 7, cluster_limit=6, metric='euclidean'):

        silhouette_max = []

        max_value = [0, 0, 0, -1]

        for e in np.delete(np.arange(0, eps_range, eps_step), 0):
            for s in range(2, min_samples_range):

                db = DBSCAN(eps=e, min_samples=s, metric=metric).fit(self.df)
                
                labels = db.labels_

                n_clusters_ = len(set(labels))

                if n_clusters_ > 1 and n_clusters_ < cluster_limit:
                    silhouette = silhouette_score(self.df, labels)
                    if silhouette > max_value[3]:
                        max_value=(e, s, n_clusters_, silhouette)
                    silhouette_max.append(silhouette)

        print(f'epsilon={max_value[0]}', 
              f'\nmin_sample={max_value[1]}',
              f'\nnumber of clusters={max_value[2]}',
              f'\nsilhouette score={max_value[3]:.3f}')


### Данные: Леденцы

Набор данных содержит оценки степени согласия респондентов с высказываниями, 
отражающими цель потребления леденцов. Всего каждый респондент выставлял оценки по 
11 высказываниям:

V1: Я потребляю леденцы, чтобы освежить дыхание

V2: Я потребляю леденцы, чтобы чувствовать себя увереннее

V3: Я потребляю леденцы, чтобы освежить полость рта

V4: Я потребляю леденцы, чтобы избавиться от неприятного вкуса во рту

V5: Я потребляю леденцы как заменитель других кондитерских изделий

V6: Я потребляю леденцы, когда мне хочется чего-нибудь сладкого

V7: Леденцы помогают мне сконцентрироваться

V8: Я потребляю леденцы, чтобы отвлечься на несколько минут и подумать

V9: Я потребляю леденцы для того, чтобы облегчить боль в горле

V10: Я потребляю леденцы для того, чтобы избавиться от заложенности носа

V11: Я потребляю леденцы для улучшения самочувствия

##### Разведочный анализ данных

In [3]:
df = pd.read_csv('data/Леденцы.dat', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   V1      338 non-null    int64
 1   V2      338 non-null    int64
 2   V3      338 non-null    int64
 3   V4      338 non-null    int64
 4   V5      338 non-null    int64
 5   V6      338 non-null    int64
 6   V7      338 non-null    int64
 7   V8      338 non-null    int64
 8   V9      338 non-null    int64
 9   V10     338 non-null    int64
 10  V11     338 non-null    int64
dtypes: int64(11)
memory usage: 29.2 KB


In [4]:
# Проверка наличия пропусков
df.isna().sum()

V1     0
V2     0
V3     0
V4     0
V5     0
V6     0
V7     0
V8     0
V9     0
V10    0
V11    0
dtype: int64

In [5]:
# Описательная статистика
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
count,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000
mean,3.544379,3.526627,3.517751,3.411243,3.207101,3.204142,3.254438,3.289941,3.603550,3.550296,3.565089
std,0.955748,0.956410,0.990150,0.992311,0.867314,0.869725,0.844107,0.821784,0.969682,0.927082,0.942042
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,5.000000,5.000000,5.000000,4.000000,3.000000,3.000000,3.000000,3.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [6]:
hist(df, title_text='Распределение оценок согласия с целями потребления леденцов')

In [7]:
# Корреляция
df.corr().style.background_gradient(cmap='Pastel2_r')

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
V1,1.000000,0.899539,0.899093,0.861459,-0.039758,-0.037705,-0.131739,-0.224225,-0.432416,-0.422820,-0.425076
V2,0.899539,1.000000,0.974008,0.918605,-0.056749,-0.051146,-0.170143,-0.243930,-0.410932,-0.408131,-0.410322
V3,0.899093,0.974008,1.000000,0.933310,-0.028481,-0.023172,-0.161636,-0.232446,-0.431511,-0.417980,-0.422759
V4,0.861459,0.918605,0.933310,1.000000,0.049004,0.053720,-0.082780,-0.146652,-0.474584,-0.462838,-0.465190
V5,-0.039758,-0.056749,-0.028481,0.049004,1.000000,0.990175,0.008874,-0.105314,-0.537177,-0.518580,-0.517738
V6,-0.037705,-0.051146,-0.023172,0.053720,0.990175,1.000000,0.038171,-0.087211,-0.554675,-0.533518,-0.532365
V7,-0.131739,-0.170143,-0.161636,-0.082780,0.008874,0.038171,1.000000,0.937104,-0.383938,-0.372838,-0.371665
V8,-0.224225,-0.243930,-0.232446,-0.146652,-0.105314,-0.087211,0.937104,1.000000,-0.257491,-0.260681,-0.262098
V9,-0.432416,-0.410932,-0.431511,-0.474584,-0.537177,-0.554675,-0.383938,-0.257491,1.000000,0.972884,0.963874
V10,-0.422820,-0.408131,-0.417980,-0.462838,-0.518580,-0.533518,-0.372838,-0.260681,0.972884,1.000000,0.991760


По матрице можно предварительно понять какие утверждения говорят примерно об одном и том же

##### Кластеризация

In [8]:
epsilon = 1.8
min_samples = 5
metric = 'euclidean'  # 'euclidean', 'cityblock', 'minkowski

dbscan = dbscan_clustering(df, eps=epsilon, min_samples=min_samples, metric=metric)

#dbscan.scatter_plot()

print(dbscan.cluster_size(), '\n')

try:
    print(f'Silhouette Coefficient: {silhouette_score(df, dbscan.dbscan.labels_):.3f}')
except:
    pass

dbscan.mean_by_cluster()

cluster
-1      6
 0    110
 1    104
 2     61
 3     57
dtype: int64 

Silhouette Coefficient: 0.693


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
cluster,,,,,,,,,,,
-1,2.333333,3.333333,3.333333,3.333333,2.000000,2.333333,3.000000,3.666667,4.333333,4.166667,4.166667
0,4.845455,4.800000,4.836364,4.681818,3.036364,3.036364,3.000000,2.990909,3.072727,3.045455,3.045455
1,2.855769,2.788462,2.721154,2.567308,2.634615,2.605769,2.778846,2.913462,4.951923,4.817308,4.865385
2,3.000000,3.000000,3.000000,3.000000,4.852459,4.852459,3.000000,2.918033,2.754098,2.754098,2.754098
3,3.000000,3.000000,3.000000,2.947368,2.947368,2.947368,4.912281,4.912281,3.000000,3.000000,3.000000


Сравним с результатами иерархического кластерного анализа:

In [9]:
link = linkage(df, 'ward', 'euclidean')

res_ = pd.DataFrame()

res_['dbscan'] = dbscan.dbscan.labels_
res_['h_clusters']  = fcluster(link, (len(set(dbscan.dbscan.labels_)) - 1), criterion='maxclust')

#  Таблица сопряженности для двух кластеризаций
tab = pd.crosstab(res_['dbscan'], res_['h_clusters'])
tab

h_clusters,1,2,3,4
dbscan,,,,
-1,4,0,2,0
0,0,110,0,0
1,104,0,0,0
2,0,0,0,61
3,0,0,57,0


In [10]:
dbscan.grid_search(eps_range=7, eps_step=0.1, min_samples_range=7, cluster_limit=6)

epsilon=1.8 
min_sample=5 
number of clusters=5 
silhouette score=0.693


Выводы:

- DBSCAN в целом схожим образом определил кластеры, однако некоторые точки, которые очень похожи на кластер 1 - были помечены как выбросы


Посмотрим какие наблюдения были отнесены к выбросам:

In [11]:
dbscan._df_result[dbscan._df_result['cluster'] == -1]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,cluster
112,2,4,4,4,2,2,2,3,5,5,5,-1
139,2,4,4,4,2,2,2,3,5,5,5,-1
177,2,4,4,4,2,2,2,3,5,4,4,-1
188,2,4,4,4,2,2,2,3,5,5,5,-1
308,3,2,2,2,2,3,5,5,3,3,3,-1
324,3,2,2,2,2,3,5,5,3,3,3,-1


Видно, что первые 4 выброса (индексы 112, 139, 177, 188) - это респонденты, которые принадлежат сразу двум кластерам из предыдущих примеров: 1) те, кто принимает леденцы для освежения полости рта. 2) те, кто принимает леденцы для лечебных целей. Остальные 2 выброса (индексы 308, 324) не так обоснованны и могут быть отнесены к кластеру про умственную активность.

Хоть дальнейшее изменение параметров не приводит к лучшим результатам, можно попробовать выделить первые 4 выброса в отдельный кластер:

In [12]:
# Смотрим получится ли кластеризовать лучше
for e in np.delete(np.arange(0, 2, 0.2), 0):
        for s in range(2, 7):

            db = dbscan_clustering(df, eps=e, min_samples=s, metric='euclidean')

            print(f'Parametrs: eps = {e}, min_samples = {s}' + '\n')
            print(db.cluster_size(), '\n')

            try:
                print(f'Silhouette Coefficient: {silhouette_score(df, db.dbscan.labels_):.3f}' + '\n')

            except:
                 continue

Parametrs: eps = 0.2, min_samples = 2

cluster
-1     14
 0     10
 1      5
 2      2
 3     49
 4      2
 5      3
 6     12
 7      4
 8      2
 9      9
 10     3
 11     2
 12     2
 13    49
 14     3
 15    15
 16    10
 17     2
 18     3
 19     6
 20     4
 21     3
 22     4
 23    37
 24    10
 25     5
 26     9
 27    52
 28     3
 29     2
 30     2
dtype: int64 

Silhouette Coefficient: 0.931

Parametrs: eps = 0.2, min_samples = 3

cluster
-1     30
 0     10
 1      5
 2     49
 3      3
 4     12
 5      4
 6      9
 7      3
 8     49
 9      3
 10    15
 11    10
 12     3
 13     6
 14     4
 15     3
 16     4
 17    37
 18    10
 19     5
 20     9
 21    52
 22     3
dtype: int64 

Silhouette Coefficient: 0.853

Parametrs: eps = 0.2, min_samples = 4

cluster
-1     48
 0     10
 1      5
 2     49
 3     12
 4      4
 5      9
 6     49
 7     15
 8     10
 9      6
 10     4
 11     4
 12    37
 13    10
 14     5
 15     9
 16    52
dtype: int64 

Silhouette C

In [13]:
epsilon = 1.8
min_samples = 3
metric = 'euclidean'  # 'euclidean', 'cityblock', 'minkowski

dbscan = dbscan_clustering(df, eps=epsilon, min_samples=min_samples, metric=metric)

print(dbscan.cluster_size(), '\n')

try:
    print(f'Silhouette Coefficient: {silhouette_score(df, dbscan.dbscan.labels_):.3f}')
except:
    pass

dbscan.mean_by_cluster()

cluster
-1      2
 0    110
 1    104
 2      4
 3     61
 4     57
dtype: int64 

Silhouette Coefficient: 0.671


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
cluster,,,,,,,,,,,
-1,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,5.000000,5.000000,3.000000,3.000000,3.000000
0,4.845455,4.800000,4.836364,4.681818,3.036364,3.036364,3.000000,2.990909,3.072727,3.045455,3.045455
1,2.855769,2.788462,2.721154,2.567308,2.634615,2.605769,2.778846,2.913462,4.951923,4.817308,4.865385
2,2.000000,4.000000,4.000000,4.000000,2.000000,2.000000,2.000000,3.000000,5.000000,4.750000,4.750000
3,3.000000,3.000000,3.000000,3.000000,4.852459,4.852459,3.000000,2.918033,2.754098,2.754098,2.754098
4,3.000000,3.000000,3.000000,2.947368,2.947368,2.947368,4.912281,4.912281,3.000000,3.000000,3.000000


Теперь есть отдельный кластер для тех, кто считает леденцы освежителями и лечебным средством. При этом остались два выброса, которые не получилось занести в кластер '4' (их также возможно выделить в отдельный кластер)

**Итог:** в данном случае все три метода кластеризации показали примерно одинаковые результаты. Иерархическая кластеризация и K-means выделяют 4 понятных и логичных кластера. Кластеризация через DBSCAN с одной стороны точнее углубляется в данные и находит случай, который игнорируется двумя предыдущими методами, с другой стороны может помечать важные наблюдения как выбросы.

### Данные: Экономика городов

Данные описывают экономические условия в 48 городах мира в 1991 году. 
Данные были собраны отделом экономических исследований банка Union (Швейцария). 
Описаны экономические условия в 48 городах мира в 1991 году. 


Число наблюдений: 48 

Названия переменных: 
- City (Город): Название города

- Работа (Work): Взвешенное среднее числа рабочих часов, сосчитанное по 12 
профессиям 

- Цена (Price): Индекс цен 112 товаров и услуг, включая арендную плату за 
жилье (значение для Цюриха взято за 100%) 

- Заработная плата (Salary): Индекс заработной платы за час работы, 
сосчитанный по 12 профессиям после налогов и вычетов  (значение для Цюриха 
взято за 100%)

##### Разведочный анализ данных

In [14]:
df = pd.read_csv('data/Econom_Cities_data.csv', sep=';', decimal=',').set_index('City', drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, Amsterdam to Zurich
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Work    48 non-null     int64  
 1   Price   48 non-null     float64
 2   Salary  48 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.5+ KB


In [15]:
# Проверка наличия пропусков
df.isna().sum()

Work      0
Price     0
Salary    0
dtype: int64

In [16]:
# Описательная статистика
df.describe()

,Work,Price,Salary
count,48.000000,48.000000,48.000000
mean,1384.958333,68.860417,-378.727083
std,2404.897007,21.784659,2027.338052
min,-9999.000000,30.300000,-9999.000000
25%,1740.750000,49.650000,12.250000
50%,1834.500000,70.500000,40.300000
75%,1972.750000,81.700000,58.700000
max,2375.000000,115.500000,100.000000


Видно подозрительные значения в строке min. Такого быть не может, поэтому нужно удалить эти выбросы:

In [17]:
indexes_for_drop = df[(df['Work'] < 0) | (df['Salary'] < 0)].index

df.drop(index=indexes_for_drop, inplace=True)

df.describe()

,Work,Price,Salary
count,46.000000,46.000000,46.000000
mean,1879.913043,70.100000,39.545652
std,174.342552,21.389177,24.757703
min,1583.000000,30.300000,2.700000
25%,1745.250000,51.775000,14.375000
50%,1849.000000,70.950000,43.650000
75%,1976.250000,81.900000,59.700000
max,2375.000000,115.500000,100.000000


In [18]:
# Проверка на выбросы
boxplots(df[['Work']])
boxplots(df[['Salary', 'Price']])

В целом, сильных выбросов не осталось, пока оставим наблюдение со значением 2375 признака Work. Перед кластеризацией также нормализуем данные, поскольку значения в колонке Work сильно отличаются от Price и Salary:

In [19]:
df_scaled = scale(df, scaler='Standard')
df_scaled.head()

,Work,Price,Salary
City,,,
Amsterdam,-0.962166,-0.212712,0.386095
Athens,-0.509827,-0.770489,-0.373488
Bogota,1.577891,-1.522069,-1.145323
Bombay,0.997969,-1.881316,-1.398517
Brussels,-0.996961,0.174896,0.447351


##### Кластеризация

In [20]:
epsilon = 1
min_samples = 5
metric = 'euclidean'  # 'euclidean', 'cityblock', 'minkowski

dbscan = dbscan_clustering(df_scaled, eps=epsilon, min_samples=min_samples, metric=metric, origin_df=df)

dbscan.scatter_plot(color='sunset')

print(dbscan.cluster_size(), '\n')
print(dbscan.mean_by_cluster())

try:
    print(f'Silhouette Coefficient: {silhouette_score(df, dbscan.dbscan.labels_):.3f}')
except:
    pass

dbscan._df_result['cluster'].sort_values()

cluster
-1     8
 0    38
dtype: int64 

                Work      Price     Salary
cluster                                   
-1       1900.375000  99.925000  61.262500
 0       1875.605263  63.821053  34.973684
Silhouette Coefficient: 0.159


City
Hong_Kong        -1
Geneva           -1
Helsinki         -1
Oslo             -1
Taipei           -1
Tokyo            -1
Zurich           -1
Stockholm        -1
Chicago           0
Caracas           0
Buenos_Aires      0
Copenhagen        0
Brussels          0
Athens            0
Bogota            0
Amsterdam         0
Houston           0
Johannesburg      0
Lagos             0
Kuala_Lumpur      0
London            0
Los_Angeles       0
Luxembourg        0
Lisbon            0
Madrid            0
Manila            0
Milan             0
Mexico_City       0
Frankfurt         0
Dusseldorf        0
Dublin            0
Bombay            0
Nicosia           0
New_York          0
Nairobi           0
Montreal          0
San_Paulo         0
Panama            0
Rio_de_Janeiro    0
Paris             0
Sydney            0
Singpore          0
Seoul             0
Tel_Aviv          0
Toronto           0
Vienna            0
Name: cluster, dtype: int64

Результаты неудовлетворительные, попробеум через grid search:

In [21]:
dbscan.grid_search(eps_range=7, eps_step=0.1, min_samples_range=7, cluster_limit=8)

epsilon=0.8 
min_sample=2 
number of clusters=5 
silhouette score=0.394


In [22]:
epsilon = 0.8
min_samples = 2
metric = 'euclidean'  # 'euclidean', 'cityblock', 'minkowski'

dbscan = dbscan_clustering(df_scaled, eps=epsilon, min_samples=min_samples, metric=metric, origin_df=df)

dbscan.scatter_plot(color='sunset')

print(dbscan.cluster_size(), '\n')
print(dbscan.mean_by_cluster(), '\n')

try:
    print(f'Silhouette Coefficient: {silhouette_score(df_scaled, dbscan.dbscan.labels_):.3f}', '\n')
except:
    pass

print(dbscan._df_result['cluster'].sort_values())

cluster
-1     4
 0    19
 1    19
 2     2
 3     2
dtype: int64 

                Work       Price     Salary
cluster                                    
-1       2051.250000   93.600000  42.375000
 0       1792.000000   77.526316  55.157895
 1       1959.210526   50.115789  14.789474
 2       1874.000000   97.950000  95.150000
 3       1625.000000  114.550000  65.150000 

Silhouette Coefficient: 0.394 

City
Hong_Kong        -1
Tokyo            -1
Taipei           -1
Stockholm        -1
Dublin            0
Amsterdam         0
Chicago           0
Brussels          0
Houston           0
Montreal          0
New_York          0
Milan             0
London            0
Dusseldorf        0
Copenhagen        0
Frankfurt         0
Paris             0
Sydney            0
Toronto           0
Vienna            0
Luxembourg        0
Los_Angeles       0
Madrid            0
Johannesburg      1
Manila            1
Kuala_Lumpur      1
Buenos_Aires      1
Athens            1
Bombay            1
Bogot

Grid search выявил, что при параметрах eps=0.8 и min_samples=2, коэффициент силуэта достигает 0.39. Это посредственный результат, но в целом, исходя из точечной диаграммы, логика разбиения есть. Как покажут дальнейшие попытки построить модель лучше, данное разбиение можно считать оптимальным.

Здесь:

- кластер 0 - плотное ядро развитых городов с высокой зарплатой и низкой рабочей нагрузкой. Включает ключевые центры Западной Европы, Северной Америки, и Австралии (Лондон, Нью-Йорк, Париж, Чикаго и др.)

- кластер 1 определяется высокой рабочей нагрузкой и очень низкой оплатой труда и низкими ценами

- кластер 2 - это изолированный кластер, характеризующийся максимальной оплатой труда, высокими ценами и средней рабочей нагрузкой (Женева, Цюрих)

- кластер 3 про города, где работают меньше остальных при высокой оплате труда, но с экстремально высокими ценами (Осло, Хельсинки)

In [23]:
# Смотрим получится ли кластеризовать лучше
for e in np.delete(np.arange(0, 1, 0.05), 0):
        for s in range(2, 4):

            db = dbscan_clustering(df_scaled, eps=e, min_samples=s, metric='euclidean', origin_df=df)

            print(f'Parametrs: eps = {e}, min_samples = {s}' + '\n')
            print(db.cluster_size(), '\n')

            try:
                print(f'Silhouette Coefficient: {silhouette_score(df_scaled, db.dbscan.labels_):.3f}' + '\n')

            except:
                 continue

Parametrs: eps = 0.05, min_samples = 2

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.05, min_samples = 3

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.1, min_samples = 2

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.1, min_samples = 3

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.15000000000000002, min_samples = 2

cluster
-1    44
 0     2
dtype: int64 

Silhouette Coefficient: -0.170

Parametrs: eps = 0.15000000000000002, min_samples = 3

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.2, min_samples = 2

cluster
-1    44
 0     2
dtype: int64 

Silhouette Coefficient: -0.170

Parametrs: eps = 0.2, min_samples = 3

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.25, min_samples = 2

cluster
-1    40
 0     2
 1     2
 2     2
dtype: int64 

Silhouette Coefficient: -0.345

Parametrs: eps = 0.25, min_samples = 3

cluster
-1    46
dtype: int64 

Parametrs: eps = 0.30000000000000004, min_samples = 2

cluster
-1    34
 0     2
 1     2
 2     2
 3     2
 4   

### Данные: Цифры

Каждая строка набора данных описывает цифру. Цифры отсканированы с ошибками. В обучающей выборке присутствует группирующая переменная - правильная цифра.

Первый этап. Сначала кластеризуем наблюдения, чтобы похожие цифры собрались в группы. Определить число кластеров и разбить цифры на кластеры. Также предложить интерпретацию для каждого кластера. 

Группирующую переменную 'A' нельзя использовать при кластеризации, 
но рекомендуется использовать ее при интерпретации кластеров.

В данных 7 переменных с именами 'B' - 'H', измеренных в номинальной шкале 
0 = линия присутствует 
1 = линия отсутствует 

Линии соответствуют черточкам на экране калькулятора

B - top horizontal, 
C - upper left vertical, 
D - upper right vertical, 
E - middle horizontal, 
F - lower left vertical, 
G - lower right vertical, 
H - bottom horizontal.  

В наборе данных 8 переменных и 500 наблюдений. По неизвестной причине в таблице данных каждый столбец присутствует дважды

In [24]:
# Загружаем данные и избавляемся от дублирующихся столбцов
df = pd.read_csv('data/digit.dat', sep=';').iloc[:, :8]

df = df.apply(lambda x: x.str.strip() if x.dtype == object else x).set_index('A')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, seven to seven
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   B       500 non-null    object
 1   C       500 non-null    object
 2   D       500 non-null    object
 3   E       500 non-null    object
 4   F       500 non-null    object
 5   G       500 non-null    object
 6   H       500 non-null    object
dtypes: object(7)
memory usage: 31.2+ KB


##### Разведочный анализ данных

In [25]:
# Проверка наличия пропусков
df.isna().sum()

B    0
C    0
D    0
E    0
F    0
G    0
H    0
dtype: int64

In [26]:
# Описательная статистика
df.describe()

,B,C,D,E,F,G,H
count,500,500,500,500,500,500,500
unique,2,2,2,2,2,2,2
top,ONE,ONE,ONE,ONE,ZERO,ONE,ONE
freq,375,299,363,363,290,416,334


Заменим текст в данных на числа:

In [27]:
mapping_dict = {'ZERO': 0,
                'ONE': 1,
                'zero': 0,
                'one': 1,
                'two': 2,
                'three': 3,
                'four': 4,
                'five': 5,
                'six': 6,
                'seven': 7,
                'eight': 8,
                'nine': 9}

df_numeric = df.replace(mapping_dict)
df_numeric

,B,C,D,E,F,G,H
A,,,,,,,
seven,1,0,1,0,0,1,0
one,0,0,1,0,0,1,0
four,0,1,1,1,0,1,0
two,1,1,1,1,1,0,0
eight,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...
five,1,1,0,1,0,1,1
seven,1,0,1,1,1,1,0
four,0,0,1,1,1,0,0


In [28]:
df_numeric.describe()

,B,C,D,E,F,G,H
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.750000,0.598000,0.726000,0.726000,0.420000,0.832000,0.668000
std,0.433446,0.490793,0.446456,0.446456,0.494053,0.374241,0.471403
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.750000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
hist(df_numeric, title_text='Гистограммы Цифр')

In [30]:
# Корреляция
df_numeric.corr().style.background_gradient(cmap='Pastel2_r')

,B,C,D,E,F,G,H
B,1.000000,0.073007,-0.116503,0.121681,0.163768,-0.037062,0.397216
C,0.073007,1.000000,-0.229322,0.227969,0.119177,0.231655,0.192881
D,-0.116503,-0.229322,1.000000,-0.206491,-0.095034,-0.036174,-0.176005
E,0.121681,0.227969,-0.206491,1.000000,-0.013265,-0.096145,0.338184
F,0.163768,0.119177,-0.095034,-0.013265,1.000000,-0.246254,0.247126
G,-0.037062,0.231655,-0.036174,-0.096145,-0.246254,1.000000,-0.100963
H,0.397216,0.192881,-0.176005,0.338184,0.247126,-0.100963,1.000000


##### Кластеризация 

In [31]:
# Дефолтные значения
epsilon = 0.5
min_samples = 5
metric = 'euclidean'  # 'euclidean', 'cityblock', 'minkowski

dbscan = dbscan_clustering(df_numeric, eps=epsilon, min_samples=min_samples, metric=metric)

print(dbscan.cluster_size(), '\n')
print(dbscan.mean_by_cluster(), '\n')

try:
    print(f'Silhouette Coefficient: {silhouette_score(df_numeric, dbscan.dbscan.labels_):.3f}')
except:
    pass

cluster
-1     110
 0      24
 1      18
 2      26
 3      35
 4       5
 5      24
 6      42
 7       8
 8      36
 9      29
 10      5
 11     11
 12      5
 13      7
 14     25
 15      5
 16     31
 17      6
 18     10
 19      6
 20      8
 21      5
 22      8
 23      5
 24      6
dtype: int64 

                B         C    D         E         F    G         H
cluster                                                            
-1       0.609091  0.545455  0.6  0.527273  0.581818  0.6  0.418182
 0       1.000000  0.000000  1.0  0.000000  0.000000  1.0  0.000000
 1       0.000000  0.000000  1.0  0.000000  0.000000  1.0  0.000000
 2       0.000000  1.000000  1.0  1.000000  0.000000  1.0  0.000000
 3       1.000000  1.000000  0.0  1.000000  0.000000  1.0  1.000000
 4       1.000000  0.000000  0.0  1.000000  1.000000  1.0  1.000000
 5       1.000000  0.000000  1.0  1.000000  1.000000  0.0  1.000000
 6       1.000000  1.000000  1.0  1.000000  0.000000  1.0  1.000000
 7       0.

Слишком много кластеров, повысим min_samples:

In [32]:
epsilon = 0.5
min_samples = 11
metric = 'cityblock'  # 'euclidean', 'cityblock', 'minkowski

dbscan = dbscan_clustering(df_numeric, eps=epsilon, min_samples=min_samples, metric=metric)

print(dbscan.cluster_size(), '\n')
print(dbscan.mean_by_cluster(), '\n')

try:
    print(f'Silhouette Coefficient: {silhouette_score(df_numeric, dbscan.dbscan.labels_):.3f}')
except:
    pass

cluster
-1     199
 0      24
 1      18
 2      26
 3      35
 4      24
 5      42
 6      36
 7      29
 8      11
 9      25
 10     31
dtype: int64 

                B         C         D         E         F         G         H
cluster                                                                      
-1       0.592965  0.477387  0.633166  0.648241  0.482412  0.698492  0.562814
 0       1.000000  0.000000  1.000000  0.000000  0.000000  1.000000  0.000000
 1       0.000000  0.000000  1.000000  0.000000  0.000000  1.000000  0.000000
 2       0.000000  1.000000  1.000000  1.000000  0.000000  1.000000  0.000000
 3       1.000000  1.000000  0.000000  1.000000  0.000000  1.000000  1.000000
 4       1.000000  0.000000  1.000000  1.000000  1.000000  0.000000  1.000000
 5       1.000000  1.000000  1.000000  1.000000  0.000000  1.000000  1.000000
 6       1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000
 7       1.000000  1.000000  0.000000  1.000000  1.000000  1.0000

Здесь:

- кластер 0 - 7 

- кластер 1 - 1

- кластер 2 - 4

- кластер 3 - 5

- кластер 4 - 2

- кластер 5 - 9

- кластер 6 - 8

- кластер 7 - 6

- кластер 8 - 9

- кластер 9 - 0

- кластер 10 - 3

*Можем оставить кластер 8 с повтором цифры 9, так как способов написания может быть два

In [33]:
# Преобразуем кластеры в числовой эквивалент
new_df = dbscan._df_result.copy()

mapping_predictions = {0: 7,
                       1: 1, 
                       2: 4, 
                       3: 5, 
                       4: 2, 
                       5: 9, 
                       6: 8, 
                       7: 6,
                       8: 9,
                       9: 0,
                       10: 3}

new_df['prediction'] = new_df['cluster'].replace(mapping_predictions)
new_df['true'] = new_df.reset_index().replace(mapping_dict).set_index(df_numeric.index)['A']

yes = new_df[new_df['prediction'] == new_df['true']]

print(f'Доля правильных ответов = {len(yes) / len(new_df)}')

Доля правильных ответов = 0.496


Исходя из признаков и выделенных выбросов, доля правильных объектов должна быть около 0.6. Несмотря на то, что алгоритм чётко выделяет все цифры, где-то в данных есть ошибки. Можем посмотреть на несовпадения:

In [34]:
new_df[new_df['prediction'] != new_df['true']].head(25)

,B,C,D,E,F,G,H,cluster,prediction,true
A,,,,,,,,,,
two,1,1,1,1,1,0,0,-1,-1,2
eight,0,1,1,1,1,1,1,-1,-1,8
six,1,0,0,1,1,1,1,-1,-1,6
eight,1,1,1,1,0,1,1,5,9,8
one,0,0,1,0,1,1,0,-1,-1,1
one,1,0,1,0,1,1,0,-1,-1,1
seven,0,0,1,0,1,1,0,-1,-1,7
zero,1,0,1,0,1,1,0,-1,-1,0
one,0,0,1,0,0,1,1,-1,-1,1


Действительно, часть меток расставлена неправильно

**Итог:** данные с цифрами лучше кластеризуются с точки зрения метрики точности через K-means, но с точки зрения здравого смысла и интерпретации - кластеризация от DBSCAN чётче и предпочтительнее на этих данных (по мнению исследователя). Выбор будет зависеть от задачи: если необходимо нарастить метрику качества - то лучше выбрать K-means, если необходимо точно идентифицировать цифру по признакам и минимизировать неопределённость - то лучше выбрать DBSCAN.